# Loan risk analysis using thew German Credit Data example

This notebook shows an example of training and running a model that classifies people described by a set of attributes as good or bad credit risks.
It is based on the <a href="https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)">German Credit Data dataset</a> from the <a href="https://archive.ics.uci.edu/">UCI</a> repository. 
The target field is an integer either Good (1) or Bad (2), where it is worse to class a customer as good when they are bad (5), than it is to class a customer as bad when they are good (1).

The demonstration uses a 6 layer neural network (NN): FC(200) --> Square activation --> FC(100) --> Square activation --> FC(1) --> Square activation

The required estimated memory is: model (140MB), input (7.34MB), output (0.26MB), and context (100MB).

We start by importing the required source packages.

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

##### For reproducibility
from numpy.random import seed
seed(1)
import numpy as np
import pandas as pd

#import tensorflow as tf
#tf.set_random_seed(seed_value)
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import h5py

Using TensorFlow backend.


### Data loading
Please refer to the dataset <a href="https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)">documentation</a> for the complete list of attributes and their description.

In [2]:
df = pd.read_csv('data/german_credit/german.data.filtered.csv', index_col=0, sep=",")
df.head()

,checking,duration,credit-hist,purpose,credit-amount,savings-account,employment-duration,installment-income-ratio,debtors-guarantors,residence-since,property,age,installment-plans,housing,num-existing-credits,job,num-liable,telephone,foreign-worker,is_good
0,A11,17,A32,A43,595,A61,A73,3,A103,4,A121,23,A143,A152,1,A173,1,A191,A201,1
1,A12,16,A34,A43,284,A62,A74,4,A101,4,A121,20,A143,A152,1,A174,1,A191,A201,2
2,A11,5,A32,A43,267,A61,A75,4,A101,4,A123,26,A143,A152,1,A173,1,A192,A201,1
3,A14,5,A34,A41,1194,A61,A75,1,A101,4,A123,62,A143,A152,4,A173,1,A191,A201,1
4,A14,5,A32,A46,924,A61,A73,1,A101,1,A123,63,A142,A152,1,A173,1,A191,A201,2


### Data preprocessing

We first convert the categorial features (in the table below) to indicator vectors. 

In [3]:
df['telephone'] = df['telephone'].replace(['A191', 'A192'], [0, 1])
df['foreign-worker'] = df['foreign-worker'].replace(['A201', 'A202'], [1, 0])
df['is_good'] = df['is_good'].replace([1, 2], [1, 0])

cat_features_list = ['checking', 'credit-hist', 'purpose', 'savings-account', 'employment-duration',
                      'debtors-guarantors', 'property', 'installment-plans', 'housing',
                     'num-existing-credits','job' ]

for f in cat_features_list:
    dummy = pd.get_dummies(df[f], prefix=f.strip())
    df = pd.concat([df, dummy], axis='columns')

final = df.drop(cat_features_list,axis=1)
print(f'data shape: {final.shape}')

data shape: (100000, 59)


Subsequently, we split every row into its target value (y) and predicates (X).

In [4]:
X = final.drop(['is_good'], axis=1)
y = final['is_good']
X.head()

,duration,credit-amount,installment-income-ratio,residence-since,age,num-liable,telephone,foreign-worker,checking_A11,checking_A12,...,housing_A152,housing_A153,num-existing-credits_1,num-existing-credits_2,num-existing-credits_3,num-existing-credits_4,job_A171,job_A172,job_A173,job_A174
0,17,595,3,4,23,1,0,1,1,0,...,1,0,1,0,0,0,0,0,1,0
1,16,284,4,4,20,1,0,1,0,1,...,1,0,1,0,0,0,0,0,0,1
2,5,267,4,4,26,1,1,1,1,0,...,1,0,1,0,0,0,0,0,1,0
3,5,1194,1,4,62,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,0
4,5,924,1,1,63,1,0,1,0,0,...,1,0,1,0,0,0,0,0,1,0


We split the dataset into the training (x_train, y_train) and test (x_test, y_test) sets and scale their features. Subsequently, we split the test set into test and validation sets.

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=5, stratify=y)

feature_scaler = MinMaxScaler()
x_train = feature_scaler.fit_transform(x_train)
x_test = feature_scaler.transform(x_test)

x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=4096, random_state=5, stratify=y_test)

For later use in HE, we save the different preprocessed datasets.

In [6]:
def save_data_set(x, y, data_type, path, s=''):
    if not os.path.exists(path):
        os.makedirs(path)
    fname=os.path.join(path, f'x_{data_type}{s}.h5')
    print("Saving x_{} of shape {} in {}".format(data_type, x.shape, fname))
    xf = h5py.File(fname, 'w')
    xf.create_dataset('x_{}'.format(data_type), data=x)
    xf.close()

    print("Saving y_{} of shape {} in {}".format(data_type, y.shape, fname))
    yf = h5py.File(os.path.join(path, f'y_{data_type}{s}.h5'), 'w')
    yf.create_dataset(f'y_{data_type}', data=y)
    yf.close()

input_output_dir = "outputs/"

save_data_set(x_test, y_test, data_type='test', path=input_output_dir)
save_data_set(x_train, y_train, data_type='train', path=input_output_dir)
save_data_set(x_val, y_val, data_type='val', path=input_output_dir)

Saving x_test of shape (15904, 58) in outputs/x_test.h5
Saving y_test of shape (15904,) in outputs/x_test.h5
Saving x_train of shape (80000, 58) in outputs/x_train.h5
Saving y_train of shape (80000,) in outputs/x_train.h5
Saving x_val of shape (4096, 58) in outputs/x_val.h5
Saving y_val of shape (4096,) in outputs/x_val.h5


### The model

The model has 6 layers: 

FC(200) --> Square activation --> FC(100) --> Square activation --> FC(1) --> Square activation

In [7]:
def square(x):
    return x ** 2

model = Sequential()
model.add(Dense(200, input_shape=(x_train.shape[1],)))
model.add(Activation(activation=square))
model.add(Dense(100))
model.add(Activation(activation=square))
model.add(Dense(1))
model.add(Activation(activation=square))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               11800     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 32,001
Trainable params: 32,001
Non-trainable params: 0
__________________________________________________

#### Model training

In [8]:
def sum_squared_error(y_true, y_pred):
    return K.sum(K.square(y_pred - y_true), axis=-1)

batch_size = 400
epochs = 5
learning_rate = 0.001

model.compile(loss=sum_squared_error,  # losses.BinaryCrossentropy(from_logits=True), =>for v2
              optimizer=Adam(lr=learning_rate),
              metrics=['accuracy'])

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              validation_data=(x_val, y_val),
              shuffle=True,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 80000 samples, validate on 4096 samples
Epoch 1/5
 - 1s - loss: 0.1584 - accuracy: 0.7812 - val_loss: 0.1313 - val_accuracy: 0.8169
Epoch 2/5
 - 1s - loss: 0.1271 - accuracy: 0.8235 - val_loss: 0.1292 - val_accuracy: 0.8242
Epoch 3/5
 - 1s - loss: 0.1246 - accuracy: 0.8269 - val_loss: 0.1268 - val_accuracy: 0.8240
Epoch 4/5
 - 1s - loss: 0.1233 - accuracy: 0.8288 - val_loss: 0.1270 - val_accuracy: 0.8218
Epoch 5/5
 - 1s - loss: 0.1216 - accuracy: 0.8300 - val_loss: 0.1256 - val_accuracy: 0.8269


For later use in HE, we save the trained model.

In [9]:
def save_weights(model, index, path):
    if not os.path.exists(path):
        os.mkdir(path)
    fname = os.path.join(path, "model_epoch_{:0>4}.h5".format(index))
    print("Saving weights to: " + fname)
    model.save_weights(fname)
    s = model.to_json()

    with open(os.path.join(path, f'model_epoch{index}.json'), 'w') as f:
        f.write(s)

save_weights(model, epochs, path=input_output_dir)

Saving weights to: outputs/model_epoch_0005.h5


In [10]:
score = model.evaluate(x_test, y_test, verbose=0)

print(f'Test loss: {score[0]:.3f}')
print(f'Test accuracy:{score[1] * 100:.3f}')

Test loss: 0.126
Test accuracy:82.438


#### Using the model for classifying cleartest data

In [11]:
    y_pred = model.predict_classes(x_test)

    f, t, thresholds = metrics.roc_curve(y_test, y_pred)
    cm = metrics.confusion_matrix(y_test, y_pred)
    print("Score:", metrics.auc(f, t))
    print("Classification report:")
    print(metrics.classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(cm)

Score: 0.7768682362551952
Classification report:
              precision    recall  f1-score   support

           0       0.73      0.66      0.69      4782
           1       0.86      0.90      0.88     11122

    accuracy                           0.82     15904
   macro avg       0.80      0.78      0.78     15904
weighted avg       0.82      0.82      0.82     15904

Confusion Matrix:
[[3145 1637]
 [1156 9966]]


### Using the model for classifying encrypted data

To run the model over encrypted samples with homomorphic encryption (HE), we first load the pyhelayers package and refer it to the directory "output/", where we saved the model and the relevant datasets.

In [12]:
import pyhelayers

Load test data and labels from the h5 file

In [13]:
with h5py.File(input_output_dir + "x_test.h5") as f:
    x_test = np.array(f["x_test"])
with h5py.File(input_output_dir + "y_test.h5") as f:
    y_test = np.array(f["y_test"])

Load a plain model

In [14]:
nnp = pyhelayers.NeuralNetPlain()
nnp.init_arch_from_json_file(input_output_dir + "model_epoch5.json")
nnp.init_weights_from_hdf5_file(input_output_dir + "model_epoch_0005.h5")
print("loaded plain model")

loaded plain model


Apply automatic optimziations

In [15]:
context = pyhelayers.DefaultContext()
optimizer = pyhelayers.HeProfileOptimizer(nnp, context)
optimizer.get_requirements().set_batch_size(16)
profile = optimizer.get_optimized_profile(False)
batch_size = profile.get_batch_size()

To reduce the memory requirements of the context, we reduce the number of rotation keys.

In [16]:
pf1=pyhelayers.PublicFunctions()
pf1.rotate=pyhelayers.RotationSetType.CUSTOM_ROTATIONS
pf1.set_rotation_steps([1,2,4,8,16])
pf1.conjugate=True
requirements = profile.requirement
requirements.public_functions=pf1

Intialize the HE context with the optimized configuration.

In [17]:
context.init(profile.requirement)
print('HE Context ready. Batch size=',batch_size)

HE Context ready. Batch size= 16


Print the HE context (w/ keys) size.

In [18]:
evalBuf=context.save_to_buffer();
print('Size',len(evalBuf)/1024/1024,'MB')

Size 100.13051319122314 MB


#### Encrypt the model

In [19]:
nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)


Object (detailed printing not implemented yet)

We use the encrypted model over batches of 16 records at a time. 

In [20]:
plain_samples = x_test.take(indices=range(0, 16), axis=0)
labels = y_test.take(indices=range(0, 16), axis=0)

Encrypt input samples

In [21]:
samples = nn.encode_encrypt_input(plain_samples)

Now we perform inference of the 16 samples under encryption 

In [22]:
predictions=nn.predict(samples)

### Plaintext results

Decrypting the final results

In [23]:
plain_predictions = nn.decrypt_decode_output(predictions)

In [24]:
print('\nclassification results')
print('=========================================')
for label,pred in zip(labels,plain_predictions):
    print('Label:',('Good' if label==1 else 'Bad.'),end=', ')
    print('Prediction:',('Bad' if pred[0]<0.5 else 'Good.'))


classification results
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Good, Prediction: Bad
Label: Bad., Prediction: Bad
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Bad., Prediction: Bad
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Good, Prediction: Bad
Label: Good, Prediction: Good.
Label: Good, Prediction: Good.
Label: Bad., Prediction: Bad
Label: Good, Prediction: Good.
